In [6]:
cd /home/liaowenjie/桌面/多模态大模型/lmms-finetune/eval/score

/home/liaowenjie/桌面/多模态大模型/lmms-finetune/eval/score


In [7]:
from builder import load_pretrained_model
import torch
import numpy as np
import os

model_path = model_base = '../../../models/Qwen-VL-Chat'
model_name = 'Qwen-VL-Chat'
model, processor, tokenizer, config = load_pretrained_model(
    model_path, model_base, model_name, device='cuda'
)


Loading qwen-vl-chat from base model ../../../models/Qwen-VL-Chat...


The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".


{'heads': 16, 'image_size': 448, 'image_start_id': 151857, 'layers': 48, 'mlp_ratio': 4.9231, 'output_dim': 4096, 'patch_size': 14, 'width': 1664}


Loading checkpoint shards: 100%|██████████| 10/10 [00:03<00:00,  2.67it/s]

Load model in torch.bfloat16


In [25]:
from transformers import AutoTokenizer, PreTrainedTokenizer
from typing import List, Tuple
def get_stop_words_ids(chat_format, tokenizer):
    if chat_format == "raw":
        stop_words_ids = [tokenizer.encode("Human:"), [tokenizer.eod_id]]
    elif chat_format == "chatml":
        stop_words_ids = [[tokenizer.im_end_id], [tokenizer.im_start_id]]
    else:
        raise NotImplementedError(f"Unknown chat format {chat_format!r}")
    return stop_words_ids
def make_context(
    tokenizer: PreTrainedTokenizer,
    query: str,
    history: List[Tuple[str, str]] = None,
    system: str = "",
    max_window_size: int = 6144,
    chat_format: str = "chatml",
):
    if history is None:
        history = []

    if chat_format == "chatml":
        im_start, im_end = "<|im_start|>", "<|im_end|>"
        im_start_tokens = [tokenizer.im_start_id]
        im_end_tokens = [tokenizer.im_end_id]
        nl_tokens = tokenizer.encode("\n")

        def _tokenize_str(role, content):
            return f"{role}\n{content}", tokenizer.encode(
                role, allowed_special=set(tokenizer.IMAGE_ST)
            ) + nl_tokens + tokenizer.encode(content, allowed_special=set(tokenizer.IMAGE_ST))

        system_text, system_tokens_part = _tokenize_str("system", system)
        system_tokens = im_start_tokens + system_tokens_part + im_end_tokens

        raw_text = ""
        context_tokens = []

        for turn_query, turn_response in reversed(history):
            query_text, query_tokens_part = _tokenize_str("user", turn_query)
            query_tokens = im_start_tokens + query_tokens_part + im_end_tokens
            if turn_response is not None:
                response_text, response_tokens_part = _tokenize_str(
                    "assistant", turn_response
                )
                response_tokens = im_start_tokens + response_tokens_part + im_end_tokens

                next_context_tokens = nl_tokens + query_tokens + nl_tokens + response_tokens
                prev_chat = (
                    f"\n{im_start}{query_text}{im_end}\n{im_start}{response_text}{im_end}"
                )
            else:
                next_context_tokens = nl_tokens + query_tokens + nl_tokens
                prev_chat = f"\n{im_start}{query_text}{im_end}\n"

            current_context_size = (
                len(system_tokens) + len(next_context_tokens) + len(context_tokens)
            )
            if current_context_size < max_window_size:
                context_tokens = next_context_tokens + context_tokens
                raw_text = prev_chat + raw_text
            else:
                break

        context_tokens = system_tokens + context_tokens
        raw_text = f"{im_start}{system_text}{im_end}" + raw_text
        context_tokens += (
            nl_tokens
            + im_start_tokens
            + _tokenize_str("user", query)[1]
            + im_end_tokens
            + nl_tokens
            + im_start_tokens
            + tokenizer.encode("assistant")
            + nl_tokens
        )
        raw_text += f"\n{im_start}user\n{query}{im_end}\n{im_start}assistant\n"

    elif chat_format == "raw":
        raw_text = query
        context_tokens = tokenizer.encode(raw_text)
    else:
        raise NotImplementedError(f"Unknown chat format {chat_format!r}")

    return raw_text, context_tokens

stop_words_ids = []
prompt = [
    {'text': "Who are you?"},
]
query = tokenizer.from_list_format(prompt)
print(query)
raw_text, context_tokens = make_context(
    tokenizer,
    query,
    max_window_size=8092,
    chat_format="chatml",
)

stop_words_ids.extend(get_stop_words_ids(
    "chatml", tokenizer
))
input_ids = torch.tensor([context_tokens]).to(model.device)
outputs = model.generate(
            input_ids,
            stop_words_ids=stop_words_ids,
            return_dict_in_generate=True,
            output_scores=True,
            output_logits=True
        )
print(outputs.scores)
print(outputs.logits)

Who are you?
(tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cu

In [13]:
prompt = [
    {'text': "Who are you?"},
]
query = tokenizer.from_list_format(prompt)
print(query)
inputs = processor(text=query, return_tensors="pt")
print(inputs) 
response, history = model.generate(**inputs, max_new_tokens=128)
print(response)

Who are you?


TypeError: 'NoneType' object is not callable

In [2]:
tokenizer

NameError: name 'tokenizer' is not defined

In [1]:
test = "- **Conclusion:** The quality of the image is fair.\n\n- **Distortion assessment:**\n  - **Noise:**  \n    - 1 instance of moderate noise affects the curtains and window details on the left side. It appears as random speckles and graininess, disrupting smoothness and slightly obscuring patterns.  \n    - 1 instance of minor noise affects the upper part of the window, introducing a mild grainy texture that reduces clarity but does not significantly alter perception.  \n\n  - **Meaningless solid color:**  \n    - 4 instances of extreme meaningless solid color distortions:  \n      - The bottom area is entirely filled with solid color, erasing visible details and textures, creating a flat, visually disruptive section.  \n      - The left side is obscured by solid color, creating a void in the composition and erasing potential objects or textures.  \n      - The top area is covered with solid color, eliminating patterns and details, disrupting continuity.  \n      - The right side is filled with solid color, erasing details and breaking the visual flow.  \n\n  - **Edge aliasing:**  \n    - 1 instance of moderate edge aliasing affects the central curtain and window edges. The edges appear jagged and exhibit stair-step patterns, diminishing smoothness and clarity.  \n\n- **Scene description:** The image depicts a window with semi-transparent curtains that allow sunlight to filter through, creating a warm, golden ambiance in the room"
print(test)

- **Conclusion:** The quality of the image is fair.

- **Distortion assessment:**
  - **Noise:**  
    - 1 instance of moderate noise affects the curtains and window details on the left side. It appears as random speckles and graininess, disrupting smoothness and slightly obscuring patterns.  
    - 1 instance of minor noise affects the upper part of the window, introducing a mild grainy texture that reduces clarity but does not significantly alter perception.  

  - **Meaningless solid color:**  
    - 4 instances of extreme meaningless solid color distortions:  
      - The bottom area is entirely filled with solid color, erasing visible details and textures, creating a flat, visually disruptive section.  
      - The left side is obscured by solid color, creating a void in the composition and erasing potential objects or textures.  
      - The top area is covered with solid color, eliminating patterns and details, disrupting continuity.  
      - The right side is filled with solid